In [1]:
import pandas as pd
%matplotlib inline
import pandas as pd
import numpy as np
import re
import math
import random
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import *

import warnings; warnings.simplefilter('ignore')
from collections import Counter
from scipy.sparse.linalg import svds



C:\Users\Zurix\AppData\Roaming\Python\Python36\site-packages\scipy\__init__.py:144: UserWarning: Numpy 1.13.3 or above is required for this version of scipy (detected version 1.13.1)
  UserWarning)


## RECOMMENDATION SYSTEM
###### HELPING FUNCTIONS

In [2]:


stemmer = PorterStemmer()

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def clean_(sen):
    
    sen = re.sub(r'[^a-zA-Z\.\:\@0-9_ ]+', '', sen) 
    sen = sen.lower()
    sen = sen.split()
#     sen = [lemmatize_stemming(x) for x in sen]
    sen = ' '.join(sen)
    
    return sen

def convertToNumber (s):
    return int.from_bytes(s.encode(), 'little')

def convertFromNumber (n):
    return n.to_bytes(math.ceil(n.bit_length() / 8), 'little').decode()


def get_events(metadata):
#     metadata = pd.read_excel(path)
    metadata['Type']=[ x.lower() for x in metadata['Type']]
        
    eventname= metadata['Event name'].fillna('')
    description = (metadata['Event time'] + metadata['Venue']+metadata['Description']+metadata['Source']+metadata['Type']+metadata['City']).fillna('')
    description_clean= list(description)

    i=0
    while(i< len(description)):
        description_clean[i]= clean_(description[i])
        i+=1
    
    #improve Description
    metadata['Description']=description_clean
    return metadata


# n=6
input_user_id=414
def get_similar_users(input_user_id,df_rating,df_events,n= 6):
    new_ratings = pd.merge(df_rating,df_events[['Event name','eventId']],left_on='eventId',right_on='eventId')
#     new_ratings = pd.merge(df_events[['Event name','eventId']],df_rating,left_on='eventId',right_on='eventId')

    #zero means that user has never voted for this item.
    rating_matrix = new_ratings.pivot(index='userId',columns='eventId',values='rating').fillna(0)
    
#     print(rating_matrix)
    knn = NearestNeighbors(metric='cosine',algorithm='brute')
    knn.fit((rating_matrix.to_numpy()))

    userid = input_user_id

    similar_user =[]

    # (excluded itself)
    value = rating_matrix.loc[ userid,:].values
    value = value.reshape(1,-1)
    print(rating_matrix.shape,'=====',np.shape(value))
    distances, indices = knn.kneighbors(value,n_neighbors=n) 
    distances=distances.flatten()
    indices=indices.flatten()

    print("Five Nearest Neighbors of user{0}:".format(userid))

    for i in range(1,len(distances)):
        similar_user.append(rating_matrix.index[indices[i]])
    print('about to return! ',similar_user)
    return similar_user



## APPLICATION DATA

###### INPUT FOR THE APPLICATION!

###### MUST COME FROM FIREBASE

In [3]:

def get_prep_data(input_user_id):
    data=pd.read_excel('activities.xls')

    df_users = pd.DataFrame(columns= ['userId','userName'])
    # df_events = get_events("activities.xls")
    df_events=get_events(data)

    m = pd.read_csv('movies/movies.csv')

    df_events['eventId'] = m['movieId']

    # df_old_events=df_events.iloc[0:int(df_events.shape[0]*0.8),:]
    # df_new_events=df_events.iloc[int(df_events.shape[0]*0.8):,:]

    df_rating = pd.read_csv("movies/ratings.csv")
    df_rating.rename(columns={'movieId':'eventId'},inplace= True)

    userRated=df_rating.loc[df_rating['userId']==input_user_id]

    df_old_events=df_events.loc[df_events['eventId'].isin(list(userRated['eventId']))] #Contains user input
    df_new_events=df_events.loc[~df_events['eventId'].isin(list(userRated['eventId']))] #Does Not contain user input
    return data,df_events,df_rating,df_old_events,df_new_events


In [4]:
# def get_frequency(df,to_check,col_name,no):
#     if int(df.shape[0]>0):
#         n=no
#         #count
#         tmp_count = Counter(list(df[to_check]))
#         tmp = pd.DataFrame.from_dict(tmp_count, orient='index')

#         #sort
#         tmp=tmp.sort_values(0,ascending=False)

#         #remove index
#         tmp.reset_index(level=0, inplace=True)

#         #rename
#         tmp=tmp.rename(columns={"index":col_name,0:"Frequency"})

#         #get Top N----
#         tmp=tmp.loc[0:n]

#         #Perc-Freq
#         sumed=sum(tmp["Frequency"])
#         tmp["Percentage of total"]=(tmp["Frequency"]/sumed)*100
#         return tmp
#     else:
#         print("DataFrame is empty")
        
# get_frequency(df_rating,'userId','userId',10)

In [5]:
# df_new_events=df_new_events.loc[df_new_events['eventId'].isin(list(df_new_events['eventId'].unique())[:100])]
# df_new_events

###### EXTRA FUNCTIONS

In [6]:
def first_element(x):
    return x[1]

def get_recommendations_pop(metadata,title,mapping): #popularity based context
    
    index = (mapping[title])
    simi = list((cosine_similarity[index]))
    indexes = list(range(len(simi)))
    simi= list(zip(indexes,simi))
    simi = sorted(simi, key=first_ele, reverse=True)
    
    
    simi = simi[1:60]
    
    #calculate vote_count and avergare before doing this
    recommendations = metadata.iloc[[i[0] for i in simi]][['title', 'vote_count', 'vote_average']]
    
    items = metadata.iloc[[i[0] for i in simi]]
    
    vote_counts = items[items['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = items[items['vote_average'].notnull()]['vote_average'].astype('float')
    
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    
    
    recommendations = items[(items['vote_count'].notnull()) & (items['vote_average'].notnull()) & (items['vote_count'] >= m)]
    
    recommendations['vote_count'] = recommendations['vote_count'].astype('int')
    recommendations['vote_average'] = recommendations['vote_average'].astype('float')
    
    recommendations['weighted_rating'] = recommendations.apply(weighted_rating,args=(C,m), axis=1)
    recommendations = recommendations.sort_values('weighted_rating', ascending=False)
#     print (recommendations.head(2))
    recommendations=recommendations[['title', 'vote_average']]
    recommendations=recommendations.head(10)
    return recommendations


def weighted_rating(x,C,m):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [7]:
#inputs to below function: input_user_id,df_rating,df_events,n= 6

## RECOMMENDATION SYSTEM
###### MAIN FUNCTION

In [16]:
input_user_id=414

#get data
data,df_events,df_rating,df_old_events,df_new_events=get_prep_data(input_user_id)


min_active=8 #other 8 users recommending new events 
rec=100  #input user top 100 history 
recommentation_number = 10 # num of recommendations to output

#main function
def system(input_user_id,df_new_events, df_old_events,df_rating,min_active_users=min_active,recent=rec , rec_num = recommentation_number):
    
    
    userRated = df_rating.loc[df_rating['userId']==input_user_id]
    # new user
    if len(userRated) < 20:
        if len(df_new_events)<=rec_num:
            rec_num = len(df_new_events) -1 
        li = random.sample(range(0, len(df_new_events)-1), rec_num)
        eid =np.array(list(df_new_events['eventId']))
        
        #return random events from new events
        return list(eid[li])
        
    app_active_users = []
    
    for n_e in range(len(df_new_events)):
        if df_new_events.iloc[n_e]['eventId'] in df_rating['eventId']:
            app_active_users.extend( list(df_rating[df_rating['eventId'] == df_new_events.iloc[n_e]['eventId'] ]['userId'])  )
    app_active_users=np.unique(np.array(app_active_users))
    
    
    #Post cold start        
    similar_users_id=[]
    if len(app_active_users)> min_active_users:
        app_active_users=np.append(app_active_users,input_user_id)

        df_subset_ratings = df_rating[df_rating.userId.isin(app_active_users)]

        similar_users_id = get_similar_users(input_user_id,df_subset_ratings, df_old_events)
        similar_users_id.append(input_user_id)
        
        #get older user data.
        df_user_rating = df_rating[df_rating['userId']==input_user_id]

        recentRatings=df_user_rating.sort_values('timestamp')['eventId'].unique()[:recent]
        df_ratings_old=df_rating.loc[df_rating['eventId'].isin(recentRatings)]

        #similar users
        df_ratings_old=df_ratings_old.loc[df_ratings_old['userId'].isin(similar_users_id)]
        df_ratings_new=df_rating.loc[df_rating['eventId'].isin(list(df_new_events['eventId']))]

        #get Recommended Events...
        df_ratings_new=df_ratings_new.loc[df_ratings_new['userId'].isin(similar_users_id)]
        df_ratings_new
        
        df_all=df_ratings_new.append(df_ratings_old,ignore_index=True)
        df_pivot=df_all.pivot(index='userId',columns='eventId',values='rating').fillna(0)
        df_pivot

        R = df_pivot.to_numpy()
        mean_ =(np.mean(R, axis = 1)).reshape(-1, 1)

        U, sigma, E = svds(R - mean_, k = 5)
        sigma = np.diag(sigma)

        dot_prod =np.dot(np.dot(U, sigma),E)
        ratings_pred = dot_prod  + mean_

        preds_matrix = pd.DataFrame(ratings_pred, columns = df_pivot.columns,index=df_pivot.index)

        # return predictions
        preds_matrix = preds_matrix[list(df_ratings_new['eventId'])]
        print(preds_matrix)

        # user_predictions = preds_matrix.loc[userID]
        events_pred=pd.unique(preds_matrix.loc[input_user_id].sort_values(ascending=False).index.to_list())[:rec_num]
        print(events_pred)

        return events_pred
    
    else:
        #Check inputs again to be sure they contain the user
        userRated=df_rating.loc[df_rating['userId']==input_user_id]
        df_old_events=df_old_events.loc[df_old_events['eventId'].isin(list(userRated['eventId']))]
        
        #new(without user rating) + old events(with user ratings)
        result=df_old_events.append(df_new_events,ignore_index=True) #get old to new.

        description = result['Description'] # Description of all
        tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 3),min_df=0, stop_words='english')
        tfidf = tf.fit_transform(description)

        cosine_similarity = linear_kernel(tfidf, tfidf)

        df_cosine_sim=pd.DataFrame(data=cosine_similarity,columns=result['eventId'],index=result['eventId'])

        # old ratings -> recent=100 by default and min rating>=4
        
        min_rating=3
        
        recent_events=df_old_events.merge(df_rating,on='eventId')
        recent_events=recent_events.loc[recent_events['rating']>=min_rating].sort_values(by='timestamp',ascending=False)['eventId'][:recent]
        
        #get df slice with predicted values
        pred=df_cosine_sim.loc[df_cosine_sim.index.isin(recent_events),list(df_new_events['eventId'])]
        
        #get top 10 predictions based on max! 0th index=>Most recommended =>9th index=>10th place on recommendation
        predictions=pred.max().sort_values(ascending=False).index[:rec_num].to_list()
#         print(predictions)
        
        return predictions
input_user_id= convertToNumber("6c598638ade072cf")



[239, 213, 276, 445, 414, 260, 480, 336, 157, 326]

In [17]:
#!flask/bin/python
from flask import request
from flask import Flask

app = Flask(__name__)

@app.route('/get_recommendations', methods=['POST'])
def create_task():
    system(input_user_id, data,df_events,df_rating,df_old_events,df_new_events)
    if not request.json or not 'id' in request.json:
        abort(400)
    recommended_events = system(input_user_id,df_new_events, df_old_events,df_rating)
    return jsonify({'event':recommended_events})

  
@app.route('/get_recommendations')
def index():
    return "Recommended Events Get Call!"

if __name__ == '__main__':
    app.run(host="0.0.0.0", port=8000)

 * Running on http://0.0.0.0:8000/ (Press CTRL+C to quit)
